In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
#@title Clonar repositiorio GitHub de BINANA

!git clone https://github.com/durrantlab/binana.git

Cloning into 'binana'...
remote: Enumerating objects: 2413, done.
remote: Counting objects: 100% (2413/2413), done.
remote: Compressing objects: 100% (847/847), done.
remote: Total 2413 (delta 1532), reused 2411 (delta 1530), pack-reused 0 (from 0)
Receiving objects: 100% (2413/2413), 6.84 MiB | 14.50 MiB/s, done.
Resolving deltas: 100% (1532/1532), done.


In [ ]:
#@title Importar dataframe

import pandas as pd

df_final_binana = pd.read_csv("/content/drive/MyDrive/TFM/T2/BINANA/df_final_binana.csv", sep =',')
df_final_binana.shape[0]

71001

In [ ]:
instancias = df_final_binana['Nombre'].tolist()
print(len(instancias))

71001


In [ ]:
#instancias

['4GCP_AIC_B_401',
 '4GCX_PRF_A_401',
 '4GCY_DUP_A_201',
 '4GDE_FDA_A_601',
 '4GDE_FDA_B_601',
 '4GDE_FDA_C_601',
 '4GDE_FDA_D_601',
 '4GE1_TSS_A_201',
 '4GE1_TSS_B_201',
 '4GE1_TSS_C_201',
 '4GE1_TSS_D_201',
 '4GEO_BOG_A_401',
 '4GEO_BOG_A_402',
 '4GEY_C8E_A_503',
 '4GF9_LP5_C_501',
 '4GF9_STE_B_201',
 '4GG1_TZD_A_601',
 '4GH0_MYS_A_302',
 '4GH8_MTX_B_201',
 '4GH8_MTX_A_201',
 '4GHD_DHY_C_403',
 '4GHD_P6G_A_402',
 '4GHD_P6G_B_403',
 '4GHD_P6G_B_404',
 '4GHD_P6G_C_402',
 '4GHD_P6G_D_402',
 '4GHE_P6G_A_402',
 '4GHE_P6G_B_403',
 '4GHE_P6G_B_404',
 '4GHE_P6G_B_406',
 '4GHE_P6G_C_402',
 '4GHE_P6G_C_404',
 '4GHE_P6G_D_402',
 '4GHE_4NC_C_403',
 '4GHF_4NC_A_402',
 '4GHF_P6G_A_403',
 '4GHF_P6G_B_405',
 '4GHF_P6G_B_406',
 '4GHF_P6G_C_404',
 '4GHF_P6G_D_403',
 '4GHG_P6G_A_402',
 '4GHG_P6G_B_403',
 '4GHG_P6G_B_404',
 '4GHG_P6G_C_402',
 '4GHG_P6G_D_402',
 '4GHG_DHY_C_403',
 '4GHH_P6G_A_402',
 '4GHH_P6G_B_403',
 '4GHH_P6G_B_404',
 '4GHH_P6G_C_402',
 '4GHH_P6G_D_402',
 '4GHH_4NC_C_403',
 '4GHH_4NC_D

In [ ]:
#@title BINANA 2

import os
import subprocess
import pandas as pd

#----- Rutas -------------------------------------------------------------------#
BASE_DIR     = "/content/drive/MyDrive/TFM/T2/TOT_ligandos_sitios/TOTAL/TOTAL_PDB"
LIGAND_DIR   = os.path.join(BASE_DIR, "ligandos_pdbqt")
RECEPTOR_DIR = os.path.join(BASE_DIR, "sitios_pdbqt_correctos")

OUTPUT_ROOT  = "/content/drive/MyDrive/TFM/T2/BINANA/BINANA_OUTPUT"
SCRIPT       = "binana/python/run_binana.py"
#-------------------------------------------------------------------------------#


os.makedirs(OUTPUT_ROOT, exist_ok=True)


errores_binana   = []
registro_binana  = []


for filename in instancia_falta:

    ligand_path   = os.path.join(LIGAND_DIR, f"{filename}_ligando.pdbqt")
    receptor_path = os.path.join(RECEPTOR_DIR, f"{filename}_sitio.pdbqt")
    output_dir    = os.path.join(OUTPUT_ROOT, filename)

    print(f"Buscando:\n  Ligando:  {ligand_path}\n  Receptor: {receptor_path}")


    if not os.path.exists(receptor_path):
        msg = f"Receptor no encontrado para {filename}, se omite."
        print(f"{msg}")
        errores_binana.append(filename)
        registro_binana.append({"instancia": filename, "estado": "error", "mensaje": msg})
        continue

    if not os.path.exists(ligand_path):
        msg = f"Ligando no encontrado para {filename}, se omite."
        print(f"{msg}")
        errores_binana.append(filename)
        registro_binana.append({"instancia": filename, "estado": "error", "mensaje": msg})
        continue

    os.makedirs(output_dir, exist_ok=True)

    cmd = [
        "python3", SCRIPT,
        "-receptor", receptor_path,
        "-ligand", ligand_path,
        "-output_dir", output_dir
    ]

    print(f"BINANA: {filename}")
    result = subprocess.run(cmd, capture_output=True, text=True)

    if result.returncode != 0:
        msg = result.stderr or result.stdout or "Error desconocido"
        print(f"Error en {filename}:\n{msg}")
        errores_binana.append(filename)
        registro_binana.append({"instancia": filename, "estado": "error", "mensaje": msg})
    else:
        print(f"Resultado guardado en {output_dir}")
        registro_binana.append({"instancia": filename, "estado": "ok", "mensaje": "Ejecutado correctamente"})

#----- Guardar logs ------------------------------------------------------------#

# Guardar errores en .txt
with open("errores_binana.txt", "w", encoding="utf-8") as f:
    for err in errores_binana:
        f.write(err + "\n")

# Guardar log detallado en CSV
df_log = pd.DataFrame(registro_binana)
df_log.to_csv("log_binana.csv", index=False)

print(f"\nLog guardado: {len(registro_binana)} instancias procesadas")
print(f"Errores: {len(errores_binana)} — Guardados en errores_binana.txt")




In [ ]:
OUTPUT_ROOT  = "/content/drive/MyDrive/TFM/T2/BINANA/BINANA_OUTPUT"

In [ ]:
import os

lista_procesados = os.listdir(OUTPUT_ROOT)
lista_fatla = [i for i in instancias if i not in lista_procesados]
print(len(lista_procesados))
print(len(lista_fatla))


70954
97


In [ ]:
import shutil
import os

sorgente = '/content/drive/MyDrive/TFM/T2/BINANA/BINANA_OUTPUT'
destinazione = '/content/drive/MyDrive/TFM/T2/BINANA_gdmail/BINANA_OUTPUT'

# Assicurati che destinazione esista
os.makedirs(destinazione, exist_ok=True)

# Sposta tutto
for nome in os.listdir(sorgente):
    src_path = os.path.join(sorgente, nome)
    dst_path = os.path.join(destinazione, nome)
    shutil.move(src_path, dst_path)


In [ ]:
#@title BINANA 2 (omitir los archivos calculados previamente)

import os
import subprocess
import pandas as pd

#----- Rutas -------------------------------------------------------------------#
BASE_DIR     = "/content/drive/MyDrive/TFM/T2/TOT_ligandos_sitios/TOTAL/TOTAL_PDB"
LIGAND_DIR   = os.path.join(BASE_DIR, "ligandos_pdbqt")
RECEPTOR_DIR = os.path.join(BASE_DIR, "sitios_pdbqt")

OUTPUT_ROOT  = "/content/drive/MyDrive/TFM/T2/BINANA/BINANA_OUTPUT"
SCRIPT       = "binana/python/run_binana.py"
#-------------------------------------------------------------------------------#


os.makedirs(OUTPUT_ROOT, exist_ok=True)


errores_binana   = []
registro_binana  = []


for filename in lista_fatla:

    ligand_path   = os.path.join(LIGAND_DIR,   f"{filename}_ligando.pdbqt")
    receptor_path = os.path.join(RECEPTOR_DIR, f"{filename}_sitio.pdbqt")
    output_dir    = os.path.join(OUTPUT_ROOT,  filename)

    if os.path.isdir(output_dir) and os.listdir(output_dir):
        msg = "Ya procesado, se omite."
        print(f"BINANA: {filename} → {msg}")
        registro_binana.append({
            "instancia": filename,
            "estado":    "skipped",
            "mensaje":   msg
        })
        continue


    if not os.path.exists(receptor_path):
        msg = f"Receptor no encontrado, se omite."
        print(f"{filename}: {msg}")
        errores_binana.append(filename)
        registro_binana.append({
            "instancia": filename,
            "estado":    "error",
            "mensaje":   msg
        })
        continue

    if not os.path.exists(ligand_path):
        msg = f"Ligando no encontrado, se omite."
        print(f"{filename}: {msg}")
        errores_binana.append(filename)
        registro_binana.append({
            "instancia": filename,
            "estado":    "error",
            "mensaje":   msg
        })
        continue

    os.makedirs(output_dir, exist_ok=True)

    cmd = [
        "python3", SCRIPT,
        "-receptor", receptor_path,
        "-ligand",   ligand_path,
        "-output_dir", output_dir
    ]

    print(f"BINANA: {filename}")
    result = subprocess.run(cmd, capture_output=True, text=True)

    if result.returncode != 0:
        msg = result.stderr or result.stdout or "Error desconocido"
        print(f"Error en {filename}:\n{msg}")
        errores_binana.append(filename)
        registro_binana.append({
            "instancia": filename,
            "estado":    "error",
            "mensaje":   msg
        })
    else:
        print(f"Resultado guardado en {output_dir}")
        registro_binana.append({
            "instancia": filename,
            "estado":    "ok",
            "mensaje":   "Ejecutado correctamente"
        })

#----- Guardar logs ------------------------------------------------------------#

# Guardar errores en .txt
with open("/content/drive/MyDrive/TFM/T2/BINANA/errores_binana.txt", "w", encoding="utf-8") as f:
    for err in errores_binana:
        f.write(err + "\n")

# Guardar log detallado en CSV
df_log = pd.DataFrame(registro_binana)
df_log.to_csv("/content/drive/MyDrive/TFM/T2/BINANA/log_binana.csv", index=False)

print(f"\nLog guardado: {len(registro_binana)} instancias procesadas")
print(f"Errores: {len(errores_binana)} — Guardados en errores_binana.txt")


⚠️ 6W8C_R16_A_405: Receptor no encontrado, se omite.
⚠️ 6W8C_D12_B_407: Receptor no encontrado, se omite.
⚠️ 6W8C_UND_B_412: Receptor no encontrado, se omite.
⚠️ 4POV_P6G_A_220: Receptor no encontrado, se omite.
⚠️ 7SVD_D12_A_1505: Receptor no encontrado, se omite.
⚠️ 7SVD_D10_A_1507: Receptor no encontrado, se omite.
⚠️ 7VD2_UND_G_102: Receptor no encontrado, se omite.
⚠️ 7VD2_UND_H_204: Receptor no encontrado, se omite.
⚠️ 7VD2_UND_C_203: Receptor no encontrado, se omite.
⚠️ 7VD2_UND_J_102: Receptor no encontrado, se omite.
⚠️ 7VSO_HP6_A_311: Receptor no encontrado, se omite.
⚠️ 7VSO_HP6_A_319: Receptor no encontrado, se omite.
⚠️ 7XS6_HP6_B_1213: Receptor no encontrado, se omite.
⚠️ 7XS6_HP6_B_1215: Receptor no encontrado, se omite.
⚠️ 7XS6_HP6_B_1216: Receptor no encontrado, se omite.
⚠️ 7XS6_HP6_A_2014: Receptor no encontrado, se omite.
⚠️ 7XS6_HP6_A_2016: Receptor no encontrado, se omite.
⚠️ 7XS6_HP6_A_2017: Receptor no encontrado, se omite.
⚠️ 6ZO8_D10_C_1114: Receptor no encont

In [ ]:
with open("/content/drive/MyDrive/TFM/T2/BINANA/errores_binana.txt", "r", encoding= "utf-8") as f:
  contenido = f.read().split("\n")

In [ ]:
contenido = [i for i in contenido if i]

In [ ]:
print(len(contenido))

97


In [ ]:
df_binana_faltas = df_final_binana[df_final_binana['Nombre'].isin(contenido)]
df_binana_faltas.shape[0]

97